<a href="https://colab.research.google.com/github/kang9366/English-Pronunciation-Classification/blob/master/English_Pronunciation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adadelta
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, MaxPool2D, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Convolution2D, BatchNormalization, Flatten, Dropout, Dense, AveragePooling2D,MaxPooling2D, Add, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import tensorflow
import librosa
import warnings
import gc
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
def load_data(paths):
    result = []
    for path in tqdm(paths):
        data, sr = librosa.load(path, sr = 16000)
        result.append(data)
    result = np.array(result)
    return result

In [ ]:
main_path = 'Data'

In [ ]:
africa_train_data = np.load(main_path +"/npy_data/africa_npy.npy", allow_pickle = True)
australia_train_data = np.load(main_path +"/npy_data/australia_npy.npy", allow_pickle = True)
canada_train_data = np.load(main_path +"/npy_data/canada_npy.npy", allow_pickle = True)
england_train_data = np.load(main_path +"/npy_data/england_npy.npy", allow_pickle = True)
hongkong_train_data = np.load(main_path +"/npy_data/hongkong_npy.npy", allow_pickle = True)
us_train_data = np.load(main_path +"/npy_data/us_npy.npy", allow_pickle = True)
test_data = np.load(main_path +"/npy_data/test_npy.npy", allow_pickle = True)

print(len(africa_train_data))
print(len(us_train_data))
print(len(hongkong_train_data))
print(len(england_train_data))
print(len(canada_train_data))
print(len(australia_train_data))

print(len(test_data))
train_data_list = [africa_train_data, australia_train_data, canada_train_data, england_train_data, hongkong_train_data, us_train_data]

1000
1000
1000
1000
1000
1000


In [ ]:
print(len(test_data))

6100


In [ ]:
X_train = np.concatenate(train_data_list, axis= 0)

In [ ]:
X_test = np.array(test_data)

In [ ]:
# train_data의 label을 생성해 줍니다.
y_train = np.concatenate((np.zeros(1000, dtype = np.int),
                          np.ones(1000, dtype = np.int),
                          np.ones(1000, dtype = np.int) * 2,
                          np.ones(1000, dtype = np.int) * 3,
                          np.ones(1000, dtype = np.int) * 4,
                          np.ones(1000, dtype = np.int) * 5), axis = 0)
y_train = to_categorical(y_train)

In [ ]:
print("X_train shape : ", X_train.shape)
print("Train size : ", X_train.shape[0])
print("Validation size : ", X_val.shape[0])
print("Test size : ", X_test.shape[0])

In [2]:
def delete(list):
  for i in list:
    del i
  gc.collect()

In [ ]:
delete([africa_train_data, australia_train_data, canada_train_data , england_train_data, hongkong_train_data, us_train_data, test_data, train_data_list])

45

## Normalization / Standardization 

In [ ]:
def minmax_scaling(data, sr = 16000, n_fft = 1024, win_length = 1024, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        mel_ = librosa.feature.melspectrogram(i[:80000], sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)
    return ((mel - mel.min())/(mel.max()-mel.min()))

In [ ]:
def std_scaling(data, sr = 16000, n_fft = 1024, win_length = 1024, hop_length = 160, n_mels = 64):
    mel = []
    for i in tqdm(data):
        mel_ = librosa.feature.melspectrogram(i[:80000], sr = sr, n_fft = n_fft, win_length = win_length, hop_length = hop_length, n_mels = n_mels)
        mel.append(mel_)
    mel = np.array(mel)
    mel = librosa.power_to_db(mel, ref = np.max)
    return ((mel-mel.mean())/mel.std())

In [ ]:
train_m = minmax_scaling(data = X_train)
test_m = minmax_scaling(data = X_test)

train_m = train_m.reshape(-1, train_m.shape[1], train_m.shape[2], 1)
test_m = test_m.reshape(-1, test_m.shape[1], test_m.shape[2], 1)

100%|██████████| 6100/6100 [01:30<00:00, 67.74it/s]


In [ ]:
train_s = std_scaling(data = X_train)
test_s = std_scaling(data = X_test)

train_s = train_s.reshape(-1, train_s.shape[1], train_s.shape[2], 1)
test_s = test_s.reshape(-1, test_s.shape[1], test_s.shape[2], 1)

100%|██████████| 6100/6100 [01:26<00:00, 70.48it/s]


In [ ]:
np.save(main_path+'/train_m.npy',arr=train_m)
np.save(main_path+'/test_m.npy',arr=test_m)

np.save(main_path+'/train_s.npy',arr=train_s)
np.save(main_path+'/test_s.npy',arr=test_s)

In [4]:
delete([train_m, test_m, train_s, test_s])

In [ ]:
#음성데이터에 맞게끔 믹스업과 랜덤이래이져를 수정하여 썼습니다.
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255):
    def eraser(input_img):
        img_h, img_w, _ = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        c = np.array([0],dtype=np.float32)
        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
class MixupGenerator():
    def __init__(self, X_train,y_train, batch_size=32, alpha=0.05, beta=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.beta = beta
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        rlambda=np.random.uniform(self.beta, self.alpha)
        rlambda = np.round(rlambda,2)
        l = np.random.beta(rlambda, rlambda, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

In [ ]:
def first_block(input_, units = 32, dropout_rate = 0.5):
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(input_)
    x = BatchNormalization()(x)
    x_res = x
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_res])
    x = AveragePooling2D()(x)
    x = Dropout(rate=dropout_rate)(x)
    
    return x

def second_block(input_, units = 64, dropout_rate = 0.5):
    x = Convolution2D(units, 1, padding ="same", activation = "relu")(input_)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x_res = x
    x = Convolution2D(units, 1, padding ="same", activation = "relu")(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding ="same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Convolution2D(units, 1, padding = "same", activation = "relu")(x)
    x = Convolution2D(units, 3, padding ="same", activation = "relu")(x)
    x = Convolution2D(units * 4, 1, padding = "same", activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x, x_res])
    x = AveragePooling2D()(x)
    x = Dropout(rate=dropout_rate)(x)
    
    return x

In [ ]:
def build_fn():
    dropout_rate = 0.3
    
    in_ = Input(shape = (X_train.shape[1:]))
    
    block_01 = first_block(in_, units = 32, dropout_rate = dropout_rate)
    block_02 = first_block(block_01, units = 64, dropout_rate = dropout_rate)
    block_03 = first_block(block_02, units = 128, dropout_rate = dropout_rate)

    block_04 = second_block(block_03, units = 64, dropout_rate = dropout_rate)
    block_05 = second_block(block_04, units = 128, dropout_rate = dropout_rate)

    x = Flatten()(block_05)

    x = Dropout(rate = dropout_rate)(x)

    x = Dense(units = 64, activation = "relu")(x)
    x = BatchNormalization()(x)
    x_res = x
    x = Dropout(rate = dropout_rate)(x)

    x = Dense(units = 64, activation = "relu")(x)
    x = BatchNormalization()(x)
    x = Add()([x_res, x])
    x = Dropout(rate = dropout_rate)(x)

    model_out = Dense(units = 6, activation = 'softmax')(x)
    model = Model(in_, model_out)
    return model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
imagegen = ImageDataGenerator(
    width_shift_range=0.04,
    height_shift_range=0.04,
    preprocessing_function=get_random_eraser())

In [ ]:
temp_y = np.concatenate((np.zeros(1000, dtype = np.int),
                        np.ones(1000, dtype = np.int),
                         np.ones(1000, dtype = np.int) * 2,
                         np.ones(1000, dtype = np.int) * 3,
                         np.ones(1000, dtype = np.int) * 4,
                         np.ones(1000, dtype = np.int) * 5), axis = 0)

## model train function

In [ ]:
def model_train(k, patience, alpha, epochs):
  for person in range(3):
    split = KFold(n_splits = k, shuffle = True, random_state = 10)
    early_stopping = EarlyStopping(patience=patience)
    idx = [k+person for k in range(0, len(X_train), 3)][:-1]
    X_train = X_train[idx]
    temp_y = y_train[idx]

    for e , (train_idx, val_idx) in enumerate(split.split(X_train, temp_y)):
    #  if e>=7: #메모리 때문에 끊어서 사용
      X_train, y_train = X_train[train_idx], y_train[train_idx]
      x_val, y_val = X_train[val_idx], y_train[val_idx]
      training_generator = MixupGenerator(X_train, y_train, batch_size=128,datagen=imagegen)()
      checkpoint = ModelCheckpoint(main_path+person+f'/check{e}.h5',save_best_only=True)
      model = build_fn()
      model.compile(optimizer = tf.keras.optimizers.Adam(alpha),
                    loss = keras.losses.CategoricalCrossentropy(),
                    metrics=['accuracy'])
      history = model.fit(training_generator, steps_per_epoch=x_train.shape[0] //128, 
                          validation_data = (x_val, y_val), epochs = epochs, callbacks=[early_stopping, checkpoint])
      gc.collect()
      print("fold :", e)
      print("*******************************************************************")

##standardization된 data로 train

In [ ]:
X_train = np.load(main_path+'/train_s.npy')
model_train(5, 10, 0.002, 30)

In [ ]:
# model_train(5, 10, 0.002, 30)
# model_train(5, 10, 0.002, 30)

## min-maxscaled된 data로 train

In [ ]:
delete(X_train)
X_train= np.load(main_path+'/train_m.npy')
model_train(5, 10, 0.002, 30)

## 테스트

### 모델 예측

In [ ]:
def predict(list, list_, n):
  early_stopping = EarlyStopping(patience=10)

  for e in range(n):
    model = build_fn()
    model.load_weights(main_path+f'/checkcheck{e}.h5')
    print("*******************************************************************")
    list.append(model.predict(X_test))
    list_.append(np.argmax(model.predict(X_test), axis = 1))
    print("*******************************************************************")

#### min-max data 예측



In [ ]:
X_test= np.load(main_path+'/test_m.npy')
pred2 = []
pred2_ = []
predict(pred2, pred2_, 10)

#### std 예측



In [ ]:
X_test = np.load(main_path+'/test_s.npy')
pred1 = []
pred1_ = []
predict(pred1, pred1_, 5)

## 6100

In [ ]:
test_ = pd.DataFrame(index = range(0, 6100), columns = ["path", "id"])
test_["path"] = [main_path + f"/open/test/{i+1}.wav" for i in range(6100)]
test_["id"] =[i+1 for i in range(6100)]
test_.head()

In [ ]:
sample_submission = pd.read_csv(main_path + "/open/sample_submission.csv")

In [ ]:
result1 = pd.concat([test_, pd.DataFrame(np.mean(pred1, axis = 0))], axis = 1).iloc[:, 1:]
result1 = pd.merge(sample_submission["id"], result1)
result1.columns = sample_submission.columns

In [ ]:
result2 = pd.concat([test_, pd.DataFrame(np.mean(pred2, axis = 0))], axis = 1).iloc[:, 1:]
result2 = pd.merge(sample_submission["id"], result2)
result2.columns = sample_submission.columns

In [ ]:
result=result1.copy()
result.iloc[:,1:]=(result1.iloc[:,1:]+result2.iloc[:,1:])/2
result

In [ ]:
result.to_csv(main_path+"/audio.csv", index = False)